In [1]:
#Load Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

#Load CSV Files
team_data = pd.read_csv('team_data.csv')
salaries = pd.read_csv('salaries.csv')

team_data = team_data.loc[(team_data['yearID'] >= 1985) & (team_data['yearID'] <= 2016)]
team_data = team_data[['yearID', 'teamID', 'Rank', 'G', 'W', 'L', 'DivWin', 'WCWin', 'LgWin', 'WSWin', 'R', 'AB', 'H', 'HR', 'RA', 'ERA', 'ER', 'FP', 'HA', 'BBA', 'E']]

salaries = salaries[['yearID', 'teamID', 'salary']]
total_salaries = salaries.groupby(['yearID', 'teamID']).sum()

total_salaries.reset_index(drop=True, inplace=True)
team_data.reset_index(drop=True, inplace=True)
total_salaries_series = total_salaries['salary']
team_data['TSalary'] = total_salaries_series
team_data['WPct'] = round(team_data['W'] / team_data['G'], 4)

salaries2 = team_data[['yearID', 'TSalary']]
season_mean = salaries2.groupby(['yearID']).mean()
season_std = salaries2.groupby(['yearID']).std()
season_mean = season_mean.reset_index()
season_std = season_std.reset_index()
series_mean = []
series_std = []
for index in range(len(team_data)):
    year = team_data['yearID'][index]
    mean = season_mean['TSalary'][year-1985]
    series_mean.append(mean)
for index in range(len(team_data)):
    year = team_data['yearID'][index]
    mean = season_std['TSalary'][year-1985]
    series_std.append(mean)

team_data['SalaryMean'] = series_mean
team_data['SalaryStDev'] = series_std
team_data['Z-Score'] = round((team_data['TSalary'] - team_data['SalaryMean'])/(team_data['SalaryStDev']), 4)

In [19]:
batting_data = pd.read_csv('Batting.csv')

In [20]:
batting_data = batting_data.loc[(batting_data['yearID'] >= 1985) & (batting_data['yearID'] <= 2016)]

In [21]:
batting_data.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
62254,aasedo01,1985,1,BAL,AL,54,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
62255,abregjo01,1985,1,CHN,NL,6,9,0,0,0,...,1.0,0.0,0.0,0,2.0,0.0,0.0,0.0,0.0,0.0
62256,ackerji01,1985,1,TOR,AL,61,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
62257,adamsri02,1985,1,SFN,NL,54,121,12,23,3,...,10.0,1.0,1.0,5,23.0,3.0,1.0,3.0,0.0,2.0
62258,agostju01,1985,1,CHA,AL,54,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
batting_data.reset_index(drop=True, inplace=True)

In [23]:
batting_data = batting_data[['playerID', 'yearID', 'teamID', 'G', 'AB', 'R', 'HR', 'RBI']]

In [24]:
batting_data

,playerID,yearID,teamID,G,AB,R,HR,RBI
0,aasedo01,1985,BAL,54,0,0,0,0.0
1,abregjo01,1985,CHN,6,9,0,0,1.0
2,ackerji01,1985,TOR,61,0,0,0,0.0
3,adamsri02,1985,SFN,54,121,12,2,10.0
4,agostju01,1985,CHA,54,0,0,0,0.0
...,...,...,...,...,...,...,...,...
40572,zimmejo02,2016,DET,19,4,0,0,0.0
40573,zimmery01,2016,WAS,115,427,60,15,46.0
40574,zobribe01,2016,CHN,147,523,94,18,76.0
40575,zuninmi01,2016,SEA,55,164,16,12,31.0


In [6]:
batting_win_shares = batting_data['RBI'] * 9.76782975e-02

In [8]:
batting_win_shares = batting_data['R'] * 9.76782975e-02

In [9]:
batting_win_shares

62254     0.000000
62255     0.000000
62256     0.000000
62257     1.172140
62258     0.000000
            ...   
102826    0.000000
102827    5.860698
102828    9.181760
102829    1.562853
102830    0.000000
Name: R, Length: 40577, dtype: float64

In [25]:
batting_data['RBI_WinS'] = batting_data['RBI'] * 9.76782975e-02
batting_onbase = batting_data['R'] - batting_data['HR']
batting_data['OnBase_WinS'] = batting_onbase * 9.76782975e-02

In [26]:
batting_data.head()

,playerID,yearID,teamID,G,AB,R,HR,RBI,RBI_WinS,OnBase_WinS
0,aasedo01,1985,BAL,54,0,0,0,0.0,0.000000,0.000000
1,abregjo01,1985,CHN,6,9,0,0,1.0,0.097678,0.000000
2,ackerji01,1985,TOR,61,0,0,0,0.0,0.000000,0.000000
3,adamsri02,1985,SFN,54,121,12,2,10.0,0.976783,0.976783
4,agostju01,1985,CHA,54,0,0,0,0.0,0.000000,0.000000


In [47]:
batting_data['RBI_WinS_Rank'] = batting_data['RBI_WinS'].rank(ascending=False, method='max')

In [48]:
batting_data.head()

,playerID,yearID,teamID,G,AB,R,HR,RBI,RBI_WinS,OnBase_WinS,RBI_WinS_Rank
0,aasedo01,1985,BAL,54,0,0,0,0.0,0.000000,0.000000,40577.0
1,abregjo01,1985,CHN,6,9,0,0,1.0,0.097678,0.000000,21935.0
2,ackerji01,1985,TOR,61,0,0,0,0.0,0.000000,0.000000,40577.0
3,adamsri02,1985,SFN,54,121,12,2,10.0,0.976783,0.976783,13545.0
4,agostju01,1985,CHA,54,0,0,0,0.0,0.000000,0.000000,40577.0


In [49]:
data = batting_data.loc[batting_data['RBI_WinS_Rank'] <= 25]

In [50]:
data

,playerID,yearID,teamID,G,AB,R,HR,RBI,RBI_WinS,OnBase_WinS,RBI_WinS_Rank
561,mattido01,1985,NYA,159,652,107,35,145.0,14.163353,7.032837,23.0
11983,belleal01,1996,CLE,158,602,124,48,148.0,14.456388,7.423551,12.0
12274,galaran01,1996,COL,159,626,119,47,150.0,14.651745,7.032837,8.0
13580,griffke02,1997,SEA,157,608,125,56,147.0,14.358710,6.739803,16.0
14472,belleal01,1998,CHA,163,609,113,49,152.0,14.847101,6.251411,6.0
14802,gonzaju03,1998,TEX,154,606,110,45,157.0,15.335493,6.349089,4.0
14824,griffke02,1998,SEA,161,633,120,56,146.0,14.261031,6.251411,19.0
15150,mcgwima01,1998,SLN,155,509,130,70,147.0,14.358710,5.860698,16.0
15359,ramirma02,1998,CLE,150,571,108,45,145.0,14.163353,6.153733,23.0
15497,sosasa01,1998,CHN,159,643,134,66,158.0,15.433171,6.642124,3.0


In [57]:
pitching_data = pd.read_csv('Pitching.csv')

In [52]:
pitching_data.head()

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,bechtge01,1871,1,PH1,NaN,1,2,3,3,2,...,NaN,7,NaN,0,146.0,0,42,NaN,NaN,NaN
1,brainas01,1871,1,WS3,NaN,12,15,30,30,30,...,NaN,7,NaN,0,1291.0,0,292,NaN,NaN,NaN
2,fergubo01,1871,1,NY2,NaN,0,0,1,0,0,...,NaN,2,NaN,0,14.0,0,9,NaN,NaN,NaN
3,fishech01,1871,1,RC1,NaN,4,16,24,24,22,...,NaN,20,NaN,0,1080.0,1,257,NaN,NaN,NaN
4,fleetfr01,1871,1,NY2,NaN,0,1,1,1,1,...,NaN,0,NaN,0,57.0,0,21,NaN,NaN,NaN


In [58]:
pitching_data = pitching_data.loc[(pitching_data['yearID'] >= 1985) & (pitching_data['yearID'] <= 2016)]

In [54]:
pitching_data.head()

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
24984,aasedo01,1985,1,BAL,AL,10,6,54,0,0,...,7.0,0,1.0,1,366.0,43,44,5.0,3.0,13.0
24985,abregjo01,1985,1,CHN,NL,1,1,6,5,0,...,1.0,0,0.0,0,109.0,0,18,5.0,1.0,3.0
24986,ackerji01,1985,1,TOR,AL,7,2,61,0,0,...,1.0,2,3.0,0,370.0,26,35,1.0,2.0,16.0
24987,agostju01,1985,1,CHA,AL,4,3,54,0,0,...,1.0,0,3.0,0,246.0,21,27,3.0,3.0,4.0
24988,aguilri01,1985,1,NYN,NL,10,7,21,19,2,...,2.0,5,2.0,2,507.0,1,49,7.0,4.0,8.0


In [ ]:
-1.59241650e+01

In [145]:
pitching_data = pitching_data[['playerID', 'yearID', 'teamID', 'G', 'IPouts', 'ER', 'ERA', 'R']]
pitching_data.head()

,playerID,yearID,teamID,G,IPouts,ER,ERA,R
0,aasedo01,1985,BAL,54,264,37,3.78,44
1,abregjo01,1985,CHN,6,72,17,6.38,18
2,ackerji01,1985,TOR,61,259,31,3.23,35
3,agostju01,1985,CHA,54,181,24,3.58,27
4,aguilri01,1985,NYN,21,367,44,3.24,49


In [144]:
team_data = pd.read_csv('team_data.csv')
salaries = pd.read_csv('salaries.csv')

team_data = team_data.loc[(team_data['yearID'] >= 1985) & (team_data['yearID'] <= 2016)]
team_data = team_data[['yearID', 'teamID', 'Rank', 'G', 'W', 'L', 'IPouts', 'DivWin', 'WCWin', 'LgWin', 'WSWin', 'R', 'AB', 'H', 'HR', 'RA', 'ERA', 'ER', 'FP', 'HA', 'BBA', 'E']]

salaries = salaries[['yearID', 'teamID', 'salary']]
total_salaries = salaries.groupby(['yearID', 'teamID']).sum()

total_salaries.reset_index(drop=True, inplace=True)
team_data.reset_index(drop=True, inplace=True)
total_salaries_series = total_salaries['salary']
team_data['TSalary'] = total_salaries_series
team_data['WPct'] = round(team_data['W'] / team_data['G'], 4)

salaries2 = team_data[['yearID', 'TSalary']]
season_mean = salaries2.groupby(['yearID']).mean()
season_std = salaries2.groupby(['yearID']).std()
season_mean = season_mean.reset_index()
season_std = season_std.reset_index()
series_mean = []
series_std = []
for index in range(len(team_data)):
    year = team_data['yearID'][index]
    mean = season_mean['TSalary'][year-1985]
    series_mean.append(mean)
for index in range(len(team_data)):
    year = team_data['yearID'][index]
    mean = season_std['TSalary'][year-1985]
    series_std.append(mean)

In [146]:
pitching_data.reset_index(drop=True, inplace=True)

In [77]:
pitching_data.iloc[0:1]

,playerID,yearID,teamID,G,IPouts,ER,ERA,R,Team_G,Loss_Shares
0,aasedo01,1985,BAL,54,264,37,3.78,44,G 1 161,G 1 1.267826


In [96]:
player = pitching_data.iloc[0:1]
player['yearID'][0]

1985

In [98]:
row = pitching_data_cut.iloc[index:(index+1)]
year = row['yearID']
year

1    1985
Name: yearID, dtype: int64

In [116]:
data = team_data_cut.loc[(team_data_cut['yearID'] == 1999) & (team_data_cut['teamID'] == 'MIL'), ['G']]
data.reset_index(drop=True, inplace=True)
data['G'][0]

161

In [147]:
pitching_data_cut = pitching_data[['yearID', 'teamID']]
team_data_cut = team_data[['yearID', 'teamID', 'G']]

In [148]:
games_total = []
for index in range(len(pitching_data_cut)):
    row = pitching_data_cut.iloc[index:(index+1)]
    year = row['yearID'][index]
    team = row['teamID'][index]
    games = team_data_cut.loc[(team_data_cut['yearID'] == year) & (team_data_cut['teamID'] == team), ['G']]
    games.reset_index(drop=True, inplace=True)
    games_played = games['G'][0]
    games_total.append(games_played)

In [149]:
pitching_data['Team_G'] = games_total

<ipython-input-149-c15bcb763fee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitching_data['Team_G'] = games_total


In [150]:
pitching_data['Team_G']

0        161
1        162
2        161
3        163
4        162
        ... 
19978    162
19979    162
19980    162
19981    161
19982    162
Name: Team_G, Length: 19983, dtype: int64

In [155]:
innings_total = []
for index in range(len(pitching_data_cut)):
    row = pitching_data_cut.iloc[index:(index+1)]
    year = row['yearID'][index]
    team = row['teamID'][index]
    IPouts = team_data.loc[(team_data['yearID'] == year) & (team_data['teamID'] == team), ['IPouts']]
    IPouts.reset_index(drop=True, inplace=True)
    innings = IPouts['IPouts'][0] / 3
    innings_total.append(innings)

In [156]:
pitching_data['Team_Innings'] = innings_total

<ipython-input-156-739852efd1ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitching_data['Team_Innings'] = innings_total


In [177]:
pitching_data['Loss_Shares'] = pitching_data['ERA'] * 1.59241650e+01 * pitching_data['G'] / pitching_data['Team_G']

In [178]:
pitching_data['Loss_Shares_per_Inning'] = pitching_data['Loss_Shares'] / (pitching_data['IPouts'] / 3)

In [179]:
pitching_data.head()

,playerID,yearID,teamID,G,IPouts,ER,ERA,R,Team_G,Team_Innings,Loss_Shares,Loss_Shares_per_Inning,LSpI_Rank
0,aasedo01,1985,BAL,54,264,37,3.78,44,161,1427.333333,20.189072,0.229421,7116.0
1,abregjo01,1985,CHN,6,72,17,6.38,18,162,1442.333333,3.762821,0.156784,15866.0
2,ackerji01,1985,TOR,61,259,31,3.23,35,161,1448.000000,19.487815,0.225728,4379.0
3,agostju01,1985,CHA,54,181,24,3.58,27,163,1451.666667,18.886255,0.313032,5891.0
4,aguilri01,1985,NYN,21,367,44,3.24,49,162,1488.000000,6.688149,0.054672,4080.0


In [182]:
qualifying_pitchers = pitching_data.loc[pitching_data['G'] >= 20]
qualifying_pitchers.reset_index(drop=True, inplace=True)
qualifying_pitchers['Loss_Shares_per_Inning_Rank'] = qualifying_pitchers['Loss_Shares_per_Inning'].rank(ascending=True, method='max')
data = qualifying_pitchers.loc[qualifying_pitchers['Loss_Shares_per_Inning_Rank'] <= 25]
data

<ipython-input-182-dfe5feb4e8f0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qualifying_pitchers['Loss_Shares_per_Inning_Rank'] = qualifying_pitchers['Loss_Shares_per_Inning'].rank(ascending=True, method='max')


,playerID,yearID,teamID,G,IPouts,ER,ERA,R,Team_G,Team_Innings,Loss_Shares,Loss_Shares_per_Inning,LSpI_Rank,Loss_Shares_per_Inning_Rank
80,goodedw01,1985,NYN,35,830,47,1.53,51,162,1488.000000,5.263821,0.019026,768.0,1.0
96,hershor01,1985,LAN,36,719,54,2.03,72,162,1465.000000,7.183568,0.029973,1279.0,24.0
238,tudorjo01,1985,SLN,36,825,59,1.93,68,162,1464.000000,6.829697,0.024835,1166.0,8.0
460,scottmi03,1986,HOU,37,826,68,2.22,73,162,1456.333333,8.074141,0.029325,1558.0,19.0
886,hershor01,1988,LAN,35,801,67,2.26,73,162,1463.333333,7.775318,0.029121,1596.0,17.0
1241,saberbr01,1989,KCA,36,787,63,2.16,74,162,1451.666667,7.643599,0.029137,1469.0,18.0
1343,clemero02,1990,BOS,31,685,49,1.93,59,162,1442.000000,5.881128,0.025757,1201.0,12.0
2007,maddugr01,1992,CHN,35,804,65,2.18,68,162,1469.000000,7.500085,0.027985,1463.0,16.0
2107,tewksbo01,1992,SLN,33,699,56,2.16,63,162,1480.000000,7.006633,0.030071,1412.0,25.0
2603,maddugr01,1994,ATL,25,606,35,1.56,44,114,1026.333333,5.447741,0.026969,1549.0,14.0


In [194]:
salaries = pd.read_csv('salaries.csv')

In [200]:
salary_info = salaries.loc[(salaries['playerID'] == 'kershcl01') & (salaries['yearID'] == 2013), ['salary']]
salary_info.reset_index(drop=True, inplace=True)
salary_info['salary'][0]

11750000

In [201]:
salaries_22 = pd.read_csv('batting_data_final.csv')

In [203]:
salaries_22.shape

(40577, 13)

In [204]:
pitching_data_2 = pd.read_csv('pitching_data_final.csv')
pitching_data_2.shape

(19983, 18)